In [82]:
from db_utils import *
% matplotlib inline

In [83]:
# Get Pre 2015-03 data from pentaho
query = """
SELECT
sum(pageviews) as n,
timestamp, 
country_iso, 
project, 
access_method
FROM staging.pentahoviews05
WHERE is_spider = 0
AND is_automata = 0
AND project RLIKE 'wikipedia'
group by timestamp, country_name, project, access_method
"""

d =  query_analytics_store(query, {})

In [84]:
str_dimensions = ['project', 'access_method', 'country_iso', 'timestamp']
d[str_dimensions] = d[str_dimensions].applymap(lambda x: x.decode("utf-8"))

In [85]:
# sampled data overlaps with hive data for 1 month (April 2015). We delete the month sampled data
d = d[d['timestamp'] != '2015-04-01']

In [86]:
# Get post 2015-03 data from hive

query = """
SET mapred.job.queue.name=priority;
SELECT
sum(view_count) as n,
CONCAT(year, '-', month, '-01') as timestamp, 
country_code as country_iso, 
project, 
access_method
FROM wmf.projectview_hourly
WHERE agent_type = 'user'
AND project RLIKE 'wikipedia'
AND YEAR = 2015
group by year, month, country_code, project, access_method;
"""

dn =  query_hive_ssh(query, 'forecasting_refresh')

In [89]:
df = pd.concat([d, dn], axis=0)

In [90]:
df.index = df['timestamp']

In [91]:
# Add checkpoint in case data gets lost
# Last Checkpoint: July 21
df.to_csv('/Users/ellerywulczyn/wmf/pageview_forecasting/checkpoint.csv')
#df = pd.read_csv('/Users/ellerywulczyn/wmf/pageview_forecasting/checkpoint.csv')

In [92]:
group_dimensions = ['project', 'access_method', 'country_iso',]
groups = df.groupby(group_dimensions)

In [93]:
cube = {}
for group in groups:
    colname = '/'.join(group[0])
    dg = group[1]
    dg.index = dg['timestamp']
    data = pd.Series(dg['n'])
    if data.sum() > 1000000:
        cube[colname] = data

In [94]:
len(cube)

2839

In [95]:
df_cube = pd.DataFrame(cube)

In [96]:
df_cube.tail()

,ab.wikipedia/desktop/US,ace.wikipedia/desktop/DE,ace.wikipedia/desktop/US,af.wikipedia/desktop/CN,af.wikipedia/desktop/DE,af.wikipedia/desktop/FR,af.wikipedia/desktop/NL,af.wikipedia/desktop/US,af.wikipedia/desktop/Unknown,af.wikipedia/desktop/ZA,...,zh.wikipedia/mobile web/NZ,zh.wikipedia/mobile web/PH,zh.wikipedia/mobile web/RU,zh.wikipedia/mobile web/SE,zh.wikipedia/mobile web/SG,zh.wikipedia/mobile web/TH,zh.wikipedia/mobile web/TW,zh.wikipedia/mobile web/US,zh.wikipedia/mobile web/Unknown,zh.wikipedia/mobile web/VN
2015-03-01,27000,268000,48000,32000,156000,54000,26000,643000,224000,528000,...,250000,78000,129000,75000,1733000,225000,47362000,8457000,1632000,211000
2015-4-01,23314,158745,55767,31707,159036,92688,30401,553269,NaN,354343,...,246991,62517,122711,70908,1611471,207956,48812559,10153578,NaN,179986
2015-5-01,43223,115412,86982,23824,147660,75855,28311,905726,NaN,639185,...,247468,61713,123301,79066,1680911,213355,50131187,11381063,NaN,187900
2015-6-01,31862,101400,62851,15260,128717,46763,28707,647306,NaN,231775,...,241866,62424,108630,68455,1649751,205275,48374828,9233813,NaN,169015
2015-7-01,10196,18076,29026,6376,40052,172220,11315,283569,NaN,128305,...,164042,46683,55253,51700,1185434,174331,33659733,4885030,NaN,132413


In [97]:
df_cube['YearMonth'] = pd.to_datetime(df_cube.index)

In [98]:
df_cube.to_csv('/Users/ellerywulczyn/wmf/pageview_forecasting/app/data/cube.csv', index = False)

NameError: name 'pd' is not defined